In [ ]:
#here i need camera should open defaulty when i press on app

To achieve the functionality of opening the camera by default with options for switching to the front camera and uploading images from the gallery, you can follow these steps. I'll provide a sample implementation using Expo for a React Native app.

### 1. **Setup Your Expo Project**

Ensure you have the necessary libraries installed:

```sh
npx expo install expo-image-picker
npx expo install expo-camera
npm install @react-native-clipboard/clipboard
```

If needed, install with legacy peer dependencies:

```sh
npm install @react-native-clipboard/clipboard --legacy-peer-deps
```

### 2. **Configure Permissions**

Ensure your `app.json` or `App.json` includes permissions:

```json
{
  "expo": {
    "name": "YourAppName",
    "slug": "your-app-name",
    "version": "1.0.0",
    "orientation": "portrait",
    "icon": "./assets/icon.png",
    "splash": {
      "image": "./assets/splash.png",
      "resizeMode": "contain",
      "backgroundColor": "#ffffff"
    },
    "updates": {
      "fallbackToCacheTimeout": 0
    },
    "assetBundlePatterns": ["**/*"],
    "ios": {
      "supportsTablet": true,
      "infoPlist": {
        "NSCameraUsageDescription": "This app needs access to your camera.",
        "NSPhotoLibraryUsageDescription": "This app needs access to your photo library."
      }
    },
    "android": {
      "permissions": ["CAMERA", "READ_EXTERNAL_STORAGE", "WRITE_EXTERNAL_STORAGE"]
    },
    "web": {
      "favicon": "./assets/favicon.png"
    }
  }
}
```

### 3. **Implement Camera and Image Picker**

Here’s an example `App.tsx` to open the camera by default with options for switching to the front camera and picking images from the gallery.

```tsx
// App.tsx
import React, { useState, useEffect } from 'react';
import {
  SafeAreaView,
  View,
  Text,
  StyleSheet,
  TouchableOpacity,
  Image,
  Alert,
} from 'react-native';
import { Camera, CameraType } from 'expo-camera';
import * as ImagePicker from 'expo-image-picker';

const App: React.FC = () => {
  const [hasPermission, setHasPermission] = useState<boolean | null>(null);
  const [camera, setCamera] = useState<Camera | null>(null);
  const [imageUri, setImageUri] = useState<string | null>(null);
  const [cameraType, setCameraType] = useState(CameraType.back);

  useEffect(() => {
    (async () => {
      const { status } = await Camera.requestCameraPermissionsAsync();
      setHasPermission(status === 'granted');

      const { status: galleryStatus } = await ImagePicker.requestMediaLibraryPermissionsAsync();
      if (galleryStatus !== 'granted') {
        Alert.alert('Permission required', 'You need to grant access to the gallery.');
      }
    })();
  }, []);

  const takePicture = async () => {
    if (camera) {
      const photo = await camera.takePictureAsync();
      setImageUri(photo.uri);
    }
  };

  const pickImage = async () => {
    let result = await ImagePicker.launchImageLibraryAsync({
      mediaTypes: ImagePicker.MediaTypeOptions.Images,
      allowsEditing: true,
      quality: 1,
    });

    if (!result.canceled && result.assets.length > 0) {
      setImageUri(result.assets[0].uri);
    }
  };

  const toggleCameraType = () => {
    setCameraType(cameraType === CameraType.back ? CameraType.front : CameraType.back);
  };

  if (hasPermission === null) {
    return <Text>Requesting permissions...</Text>;
  }
  if (hasPermission === false) {
    return <Text>No access to camera or gallery</Text>;
  }

  return (
    <SafeAreaView style={styles.container}>
      <View style={styles.cameraContainer}>
        <Camera
          style={styles.camera}
          type={cameraType}
          ref={(ref) => setCamera(ref)}
        />
        <View style={styles.buttonContainer}>
          <TouchableOpacity style={styles.button} onPress={toggleCameraType}>
            <Text style={styles.buttonText}>Flip Camera</Text>
          </TouchableOpacity>
          <TouchableOpacity style={styles.button} onPress={takePicture}>
            <Text style={styles.buttonText}>Take Picture</Text>
          </TouchableOpacity>
          <TouchableOpacity style={styles.button} onPress={pickImage}>
            <Text style={styles.buttonText}>Pick Image from Gallery</Text>
          </TouchableOpacity>
        </View>
      </View>
      {imageUri && <Image source={{ uri: imageUri }} style={styles.image} />}
    </SafeAreaView>
  );
};

const styles = StyleSheet.create({
  container: {
    flex: 1,
    justifyContent: 'center',
    alignItems: 'center',
    backgroundColor: '#fff',
  },
  cameraContainer: {
    flex: 1,
    width: '100%',
    justifyContent: 'flex-end',
  },
  camera: {
    width: '100%',
    height: '100%',
  },
  buttonContainer: {
    flexDirection: 'row',
    justifyContent: 'space-around',
    marginBottom: 20,
  },
  button: {
    backgroundColor: '#007bff',
    padding: 10,
    borderRadius: 5,
  },
  buttonText: {
    color: '#fff',
    fontSize: 16,
  },
  image: {
    width: 300,
    height: 300,
    marginTop: 20,
  },
});

export default App;
```

### Summary

1. **Setup**: Install the necessary packages and configure permissions in `app.json`.
2. **Implementation**: Use the Expo Camera and Image Picker to allow taking pictures, flipping the camera, and picking images from the gallery.
3. **Permissions**: Ensure all permissions are requested and handled properly.

With this setup, when you run your app, it should default to opening the camera with options to flip between the front and back cameras, and you should be able to select an image from the gallery as well.